In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[211]:


""" Basic example showing samples from the dataset with bounding boxes and 2 hands"""

from __future__ import print_function, unicode_literals
import PIL
import pickle
import os
import numpy as np
import scipy.misc
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as patches

# chose between training and evaluation set
set = 'training'
#set = 'evaluation'

# load annotations of this set
with open(os.path.join(set, 'anno_%s.pickle' % set), 'rb') as fi:
    anno_all = pickle.load(fi)
images_cropped = []

#Example for 10 images
for sample_id in range(len(anno_all)):
    one_hand = False
    anno = anno_all[sample_id]
    image = scipy.misc.imread(os.path.join(set, 'color', '%.5d.png' % sample_id))

    # get info from annotation dictionary
    kp_coord_uv = anno['uv_vis'][:, :2]  # u, v coordinates of 42 hand keypoints, pixel
    kp_visible = (anno['uv_vis'][:, 2] == 1)  # visibility of the keypoints, boolean

    # Visualize data
    #fig = plt.figure(figsize=(25,25))
    #ax1 = fig.add_subplot('221')
    #ax1.imshow(image)
    #ax1.plot(kp_coord_uv[kp_visible, 0], kp_coord_uv[kp_visible, 1], 'ro')
    
    #keypoints of hand1
    hand1 = kp_coord_uv[:21]
    hand1v = kp_visible[:21]
    hand2 = kp_coord_uv[21:]
    hand2v = kp_visible[21:]
    
    if(len(hand1[hand1v,0]) <= (len(hand2[hand2v,0]))):
        one_hand = True
        hand1 = hand2
        hand1v = hand2v

    #bounding box edges
    x1,y1 = max(hand1[hand1v,0]), max(hand1[hand1v,1])
    x2,y2 = min(hand1[hand1v,0]), min(hand1[hand1v,1])
    
    #unpadded width and height
    w = abs(x1-x2)
    h = abs(y1-y2)
    
    #padding 10% hand 1 (Keypoints lie IN the hand -> Bounding box needs padding to cover whole hand)
    pad_x1 =x1+ 0.1 * w
    pad_y1 =y1+ 0.1 * h
    pad_x2 =max(0,(x2 - 0.1 * w))
    pad_y2 =max(0,(y2 - 0.1 * h))


    #bbox width and height
    w = abs(pad_x1-pad_x2)
    h = abs(pad_y1-pad_y2)
    
    if h < w:
        h = w
    else:
        w = h
    if(w>0):
        if one_hand:
            cropped_image = image[int(round(pad_y2)):int(round(pad_y2))+int(round(h)) , int(round(pad_x2)):int(round(pad_x2))+int(round(w)), :]
        else:
            cropped_image = image[int(round(pad_y2)):int(round(pad_y2))+int(round(h)) , int(round(pad_x2)):int(round(pad_x2))+int(round(w)), :]
       # ax1.imshow(image[int(round(pad_y2)):int(round(pad_y2))+int(round(h)) , int(round(pad_x2)):int(round(pad_x2))+int(round(w)), :])
        #print(cropped_image.shape)
        cropped_image = PIL.Image.fromarray(cropped_image)
        images_cropped.append([image])
    #plt.show()




/home/linux/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


In [2]:
import numpy as np
import pandas as pd

In [3]:
dataF = pd.DataFrame((images_cropped))
dataF.to_csv(path_or_buf="dataset.csv", index=False)